<a href="https://colab.research.google.com/github/icivi/Torch_Transformer_hunks/blob/main/%D0%9C%D0%B0%D0%BA%D1%81%D0%B8%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5_%D0%BA%D0%BE%D0%BB%D0%B8%D1%87%D0%B5%D1%81%D1%82%D0%B2%D0%BE_%D1%82%D0%BE%D0%BA%D0%B5%D0%BD%D0%BE%D0%B2_%D0%BD%D0%B0_%D0%B2%D1%85%D0%BE%D0%B4%D0%B5_%D0%B8_%D0%9F%D0%BE%D0%B4%D1%81%D1%87%D0%B8%D1%82%D0%B0%D1%82%D1%8C_%D0%BA%D0%BE%D0%BB%D0%B8%D1%87%D0%B5%D1%81%D1%82%D0%B2%D0%BE_%D1%82%D0%BE%D0%BA%D0%B5%D0%BD%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.9 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, AutoModelForCausalLM, AutoModelForSequenceClassification
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
name = "microsoft/prophetnet-large-uncased-squad-qg"
name = "Narrativa/mT5-base-finetuned-tydiQA-question-generation" # отстой
# name = "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli"
name = 'TheBloke/Llama-2-13B-chat-GPTQ'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(name, use_fast=True, device_map="auto") # 'T5TokenizerFast' object has no attribute 'to'

## Посмотреть файлы

In [ ]:
!git lfs install
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/{name} {name} # Просто пример clone without large files \:

Git LFS initialized.
Cloning into 'TheBloke/Llama-2-13B-chat-GPTQ'...
remote: Enumerating objects: 561, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 561 (delta 10), reused 0 (delta 0), pack-reused 540
Receiving objects: 100% (561/561), 593.27 KiB | 26.97 MiB/s, done.
Resolving deltas: 100% (250/250), done.


## Посмотреть Максимальное количество токенов на входе, vocab_size
и много чего...

In [ ]:
tokenizer

LlamaTokenizerFast(name_or_path='TheBloke/Llama-2-13B-chat-GPTQ', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True)}, clean_up_tokenization_spaces=False)

In [ ]:
len(tokenizer)

32000

In [ ]:
with open(f"{name}/tokenizer.json") as f:
    txt = f.read()[:700]
    # print(txt)
txt

'{\n  "version": "1.0",\n  "truncation": null,\n  "padding": null,\n  "added_tokens": [\n    {\n      "id": 0,\n      "content": "<unk>",\n      "single_word": false,\n      "lstrip": false,\n      "rstrip": false,\n      "normalized": true,\n      "special": true\n    },\n    {\n      "id": 1,\n      "content": "<s>",\n      "single_word": false,\n      "lstrip": false,\n      "rstrip": false,\n      "normalized": true,\n      "special": true\n    },\n    {\n      "id": 2,\n      "content": "</s>",\n      "single_word": false,\n      "lstrip": false,\n      "rstrip": false,\n      "normalized": true,\n      "special": true\n    }\n  ],\n  "normalizer": {\n    "type": "Sequence",\n    "normalizers": [\n      {\n        "type": "'

In [ ]:
text = "Подсчитать количество токенов: "

## Подсчитать количество токенов

In [ ]:
len(tokenizer.tokenize(text))

12

In [ ]:
tokenizer.tokenize(text)

['▁Под',
 'с',
 'чита',
 'ть',
 '▁коли',
 'чество',
 '▁то',
 'к',
 'ен',
 'ов',
 ':',
 '▁']

туда

In [ ]:
embed = tokenizer(text, return_tensors="pt").to(device)
embed

{'input_ids': tensor([[    1, 20266, 29935, 18790,  1413, 13142, 22339,  2721, 29951,   656,
           516, 29901, 29871]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

обратно

In [ ]:
tokenizer.decode(embed['input_ids'][0], device = device)

'<s> Подсчитать количество токенов: '

## Максимальное количество токенов на входе
c загрузкой всей модели, см график занятости диска справа

In [ ]:
%cd {name}
!git lfs pull

/content/TheBloke/Llama-2-13B-chat-GPTQ


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(name).to(device)

RuntimeError: ignored

In [ ]:
print(getattr(model.config, 'to_json_string'))
print(getattr(model.config, 'to_json_file'))
print(getattr(model.config, 'update'))

In [ ]:
for attr in dir(model.config):
    print(attr, getattr(model.config, attr) if attr not in ['to_json_string', 'to_json_file', 'update'] else None)

In [ ]:
getattr(model.config, 'd_model')

In [ ]:
for attr in dir(model):
  if attr.startswith("get"):
    print(attr, )

In [ ]:
model.get_memory_footprint()/(1e9)

In [ ]:
for attr in dir(model):
    print(attr)

In [ ]:
model

In [ ]:
getattr(model, 'transformer')

In [ ]:
print("Максимальное количество токенов на входе: ", model.config.max_position_embeddings)

Всегда удобно найти Модель Torch или Keras на HuggingFace и посмотреть

In [ ]:
generato = model.generate(embed['input_ids']).to(device)
result = tokenizer.decode(generato[0], device = device)
result

In [ ]:
pipe = pipeline(model=name, tokenizer=tokenizer, device=device)

In [ ]:
pipe(text)

In [ ]:
if 1:
    pipe = pipeline("text2text-generation", model=name, tokenizer=tokenizer, device=device)
    # generator = pipeline('text-generation', model=name, tokenizer=tokenizer) # у меня он и так ранее качал в "cuda"

In [ ]:
temperature=0.9
top_p=0.95
pipe(text, max_new_tokens = 40, temperature = temperature, top_k=60, top_p=top_p, do_sample=True,)

### половина огромного, (сейчас это 8806 токенов) :

In [ ]:
# @title
text = '''
• Лучше всего он может делать заполнение, маскирование.
• !!! Он может разметить Датасет
	○ https://github.com/PanQiWei/AutoGPTQ/blob/main/examples/evaluation/run_sequence_classification_task.py
	○ хоть 1000 классов создавай на любой вкус, включая Высокоуровневые)
• с температурой 0.9 Он же супер Гениальный Генератор Идей. Он же берет и делает несколько иначе.
• с температурой 0.9  Он так же генерирует новый продукт. Он же берет и делает несколько иначе.
• температурой 0.02 Он - Обработчик. Всегда одинаково обработает
• Да, он творчески генерирует. Он может предложить необычное решение. Даже в Коде. Только потом надо подправить.
• он хорошо понимает. Делает заголовки. То есть удалить Декодер и прикрепить другую нейронку.
• Он может вести цепочку. Последовательность.
• Да, он творчески генерирует. Но он может забирать НОРМАЛЬНОЕ из базы, а не генерировать. выбирать.
• Сам себе генерирует память (писать короткий блок памяти). По ходу это просто Заголовок. И саммари. Который должен возникать в команде чаще.
• Это же мой AI блок! Он может делать конволюшн - исчезающую - Память - Это просто Заголовок. И саммари.
• Вырезать из текста участки и заметить датасет. Но обучать точно НИКАК НЕТ ВОЗМОЖНОСТИ!!!
• Преобразовать мои трени  человеческий вид. Заголовки.
• Когда я исследую, он может выбирать из Треней и освежать память.
• Поддерживать мою целостность.
• Немного захватывать меня, мой мозг к теме.
• Куски кода для треней, как идеи.
• Он начинает писать треню за меня. он видит хронологию: 28 июля 2023 г. и пишет продолжение.

• Сделать из них техзадание.
• Подкинуть идею фреймворка для кода. Псевдокод, но не код.

• Если ты дашь ему функцию (шаблон, псевдокод) как отдельный блок файл .py - он его заполнит (шумно, плохо.) это не годится'''

In [ ]:
# @title
text = '''
вместо Google Интернета - Дорога Кино Игра Сказка.

Вау! Сказка, Счастье, Жизненный Путь Людям - Я уже это Закодил!

Я никогда не был так близко к Удаче!
это Алисой - "D:\One mp3 1 сек 2\тема - деньги\Что он делает. Идеи. Продукты\08_15 10_56 Y Вау как просто и Грандиозно. Дорога Кино Игра Сказка вместо Google Интернета!.mp3"
а ну ка для начала и старта из всех отзывов этих датасетов выбери мне
1. позитив.
2. любовь.
3. заработать.
4. творчество.

Бац. Я уже Загорелся! Вау как интересно! А если пойти дальше?
а теперь вот тебе огромные тексты, выбери из них.
А теперь вот тебе видео, выбери из них.
А теперь вот тебе мир и примеры жизни людей. Выбери теперь из этого!
Ведь код у меня уже готов! Все нейронки собраны! Теперь только собери всё и ты в Дамках



дай мне пожалуйста лучшие выдержки, которые соответствуют этим классам, которые я уже умею делать из видео, из Ютюба, из кино или от блогеров!
мне сейчас это надо!
мне надо мир, который заточен под мою текущую потребность! мне нужна светлая Ясная дорога сейчас, а точки я всё время буду тебе сейчас давать, а ты мне дорогу! Ведь Я не знаю Что делать! Ведь Никто не знает что делать!
А теперь Люди прозрели! У них есть Гениальный Подсказчик, который не просто скинет им Видео и Тесты, как и Куда идти, А Выложит и Покажут им Охуенную Сказку Дорогу Историю, что им сейчас делать! Вот он, Освещённый путь, Диснейленд, вот она игра, вот оно развлечение! Вот оно счастье, Ведь я теперь знаю, к чему я иду, сколько идти и я Примерно вижу, что на самом деле меня ждёт! Ведь это словно сказка, выложенная GPT3 или еще каким готовым генератором или GAN! Сделай это по Agile, сначала что то простенькое, дай людям. Какая то простая подсказка. Просто цифра. Потом добавим картинку. потом Три Дэ мир. потом голоса. Потом подключим живых людей сюда. Да! Люди прямо через инет будут Подрисовывать и предлагать Гармонию Вариант моего Жизненного пути Сказки!

Типа Google, но это Наглядный мир - история кино сюжет сценка под меня, Мир моей дороги. Мир, который возможно сыграют и покажут люди. Возможно даже в Метавселенной или GTA - Гэ тэ А. может добавим музыку, звуки леса или счастливый звонкий смех, добавим всегда говорить шутками и никогда не как сухарь, а всегда развлекающий, весёлый и озорник. И я вижу только то, что мне нравится, только то, где я счастлив, ведь я всё время добавляю Ключевые точки притяжения и бац - Дорога Игра ДиснейЛенд весь пересобрался, я добавил персонажей, девушек, друзей, любовь и стало еще интереснее! Смотри как просто и Грандиозно! Все нейронки собраны! Теперь только собери всё постепенно выкладывая людям по Agile и ты в Дамках.
'''

In [ ]:
# @title
text = '''
Трени - Разум, личность, и Уникальный продукт и Твой Огромный рычаг, Инструмент (готовый Алгоритм) и очень умного - легко, быстро и уже всё есть.

Твои Тренинги таят Огромный бриллиант БЫСТРО.

1. Берешь все свои тренинги.
2. Делаешь через нейронку Сафари.
3. А так же берёшь названия файлов (названий так много, что они сами по себе мощный тренинг) и название каталогов а так же добавь к нему общие описания (напиши вручную к этому всему каталогу, что хочешь, чего добиваешься, любое обобщение)
4. Модулируешь это другими потоками и обучаешь дальше, без учителя, награждая и создавая потоки.
5. Награждай и создавай побольше потоков по типу один канал создаёт связанный фрейм (кадр, данное, привязанную картинку например) в другом. с алгоритмами мышления, личности, живого, разума, осознанность себя

Секрет вот в чём: Сафари - не совсем плоское. Это что то среднее от человечество. Это и надо промодулировать потоками, 100D, будет - добиться глубины.

Получаешь
	○ разум, .
	○ личность, .
	○ алгоритмы мышления,.
	○ осознанность себя, .
	○ свою личность, .
	○ тренинги знают что тебе надо.
	○ тренинги знают что делать дальше.
	○ тренинги знают куда рыть конкретно.
	○ тренинги твоя память и огромная поддержка.
	○ тренинги это будто твоя семья, твоя живая опора, монолит, к которому не надо снова и снова поднимать, освежать.
	○ свою карту мозга, .
	○ особую штуку памяти и .
	○ мозга вместо письма и инцеклопедий (живая голова, действие живое-продукт живой, а не записи.


Твои Тренинги таят Огромный бриллиант БЫСТРО.

Понимашь, Это выгрузка мозга, это как бы огромная редкая конвертация мышления в речь. И их очень много, это огромный РЕДКИЙ УНИКАЛЬНЫЙ датасет.
Понимаешь, одних названий так много, что они сами по себе мощный тренинг и мощный датасет и мощный Поток для 100D.
Это особый плод, а как известно человечество скушает всё.
Просто надо это как то слить в одно, конвертировать, преобразовать.
Возможно очень, что это личность, голова, живой продукт, производитель, алгоритм решения, допустим для продвижения от плоской нейронки Одного действия в Цепочку Нейронок, в цепочку действий.
Надо просто придумать, что на них натравить, надо придумать, во что это вылить. Как бы рутина сделана, как бы сняты снимки с мозга, как бы нейролинком считался мозг, надо теперь просто обработать это нейронкой и будет Грандиозный мощный продукт, Инструмент, Действо и возможно целая живая страна только на основе треней. ведь все записи - это даже более чем мозг, ведь мозг людей даже в чём то уступает твоим тренингам из за своей несобранности, размытости.

Твои Тренинги это даже больше чем мозг и личность, в чём то больше чем живое. тренинги таят Огромный бриллиант.
'''

In [ ]:
# @title
text = '''
расширение мозга.

цель: текст и речь устарели. А видео и запись речи - слишком долго, растянуто, не динамично, шумно и хаотично. И всё это теряется, рассыпано, шумно и хаотично. быстро загружать раскалённое облако мышления и памяти. Также цель - целостность мышления, Полное восстановление живого действа мышления без потери большого множества квантовых частиц облака. А не только восстановление толстого дерева переднего мозга и мышления.
• тему, .
• облако мышления, .
• большой космический поток мозга, .
• квантовый поток / облако мозга (широкий рассеяный, полу - упорядоченный).

расширение мозга это
	○ память. а память и мышление - это ещё и бульён, и состояние.
	○ это действие .
	○ это вектор (от чего к чему, зачем вообще, тема).

как расширение мозга выглядит:
	визуально.
	комната, зашёл - всё ожило, заговорило, динамически меняющаяся анимация на стенах.
	слышно.
	переживается эмоция.
	объёмно.
	живая картина, сцена.

для расширение мозга использовать:
	нейралинк Маска,.
	типа социальной сети, мессенджера, платформы, института, контакт сайт-фриланса, интерактивное - то есть участие многих людей одновременно.
	делегирование, разделение по операциям и по подтемам другим людям.
	использовать Искусственный интеллект.
	Нейросеть генерирует комнату - действо исходя из того, что происходит и предсказывая, генерируя.
	Предсказатель следущего шага и генератор вариантов от среднего человеческого, как в GPT.
	Всё держит в 100D как Berta, но и человек все держит в объемном видео, плюс объемном звуке, память, внутренний голос, фантазию, мозговые предсказания, эмоции.
	использовать интернет, .
	экран, .
	3D, .
	возможно живая голова, .
	использовать эмоции, .
	динамичесий бульён медиаторов (эндорфин, дофамин, серотонин).
	речь,.
	звуки.
	видео.
	10 датчиков считывания с тела, вот и память и мышление.
	излучатели, типа динамиков, но только  для тела и мозга, внешние волновые и импульсные электро-контакты к телу и к голове.
	Маску и очки виртуальной реальности.

дальнейшее развитие речи текста письма?
снимок мозга?
комната?
'''

In [ ]:
# @title
text = '''
Асистент - Он интерактивный. Пристройка к ИИ. Позже?
D:\One mp3 1 сек 2\тема - деньги\Что он делает. Идеи. Продукты\Асистент - Пристройка к ИИ. Позже. Алисой.mp3
Кто он, общее для многих его:

Общее для многих из его вариаций:

!! Он Простой, но делает 1000 вещей вовремя и в помощь - получится сверхрезультат! это бомба! легко и правильно и свойственно нейронке!
Айробик 2, вот у меня тысячи записей, но вылетают из головы, как ты думаешь, что можно сделать.

Он интерактивный. Он как бы в ожидании постоянно. Он здесь и сейчас. Он жив событиями, его ценность в присутствии! Он нужен постоянно. Пусть делает что то простое, но он всегда со мной. Чуть что я к нему. и чуть что он обязательно что то скажет, покажет, напишет, прозвучит. Он адаптируется под ситуацию.

Хорошо, если он не пишет, а заменит письмо. Он заменит мышку и клавиатуру. Он избавит от необходимости смотреть рутинно в экран. именно рутинно. Например если видео, он его переберёт и выдаст кратко и в двух словах, больше заголовками, как самари сначало. Я вынужден прижаться к экрану сейчас, к всяким там блокнотам и записям, ко всяким директориям, но он избавляет меня от этого. Он сам всё разложит по полочкам и быстро обращается к любой структуре, к каталогам к дереву.

Его удобно регулировать. сказать тише, будь активнее. Ему можно давать больше притяжения. например давай ближе к продажам. Или давай ближе к коду, к готовому продукту или живому организму.

А по необходимости раскрывает тему, но всё именно под меня. Например если я решил посмотреть руководство по нейронкам, у меня возник вопрос, я с ним поговорил. То есть как бы сидит опытный программист. Например если бы я зашел куда то к нейронщикам. для меня открылись очевидные вещи. Это не то чтобы слишком широко и рассыпано, это как бы главное, очевидное, узко по теме. Его можно сужать. Говорить, давай ближе вот к этим точкам будем центрироваться. Он понимает, на чем застряли недавно и что важно продолжить. что тут главное, что особенно важно и что между точками притяжения.

Нейронке очень удобно быть средним специалистом, мастером и кодером из обученных берт и GPT3. Средний человек,  но на тему. С уклоном на новое, уже лучшее, он всё самое оптимизированное в интернете подобрал, а не кривое и сырое. Можно задать ему будь точнее или пофантазируй, давай предположения и творчество.

Он угадывает где мы, в городе или утро. Он наблюдает за всем, что происходило и запоминает. так он знает, о чем мы сейчас и что было час и два назад. он помнит что было вчера вечером.

Он поддерживает мой квантовый мозг, сохраняет космический поток мышления. он быстро и гибко переучивается под поток моих размышлеий и мгновенно исполнит попробует пока я думаю. Я говорю, а он уловил и уже что то попробовал закодить. Он тут же полез в инет, собрал информацию по теме и стал в этом мастером и тут же сделал несколько попыток.

Я ему говорю - там уже есть нейронки распознавания голоса. Создай да получше. Смотри как это под него четко заточено. Вот тебе и простой текст и все о цифрах и всё  коде. Ему это просто. Это его данные. И вот тут ты как раз на нём сидишь и придумаешь юнитов и цивилизацию. А он всё снова и снова вставляет в себя новые блоки подобны распознаванию голосу, а завтра переднюю память, а завтра внешнюю память и индексы, потому что ты в него снова и снова внедряешь юнитов и их варианты действий, включая всяких там оптимизаторов. Сам посуди, умная нейронка так сильно заоптимизирует любой блок до идеала почти, и ей свойственно и несложно добавить в себя тысячи блоков за один присест. И вот уже вскоре он быстро угадывает что мне надо. И вот уже вскоре он мгновенно работает с интернетом и уже советует сам, что можно добавить в себя и анализирует новые обученные нейронки на huggingface и вставляет в себя и снова его рабочие подхватывают эти блоки и находят что то новое, гибриды и эффективности. И вот он уже коннектор человек интернет и человек продукт, человек - готовый сервис.


Получается он и модуль цивилизации, ведь на основе него я буду добавлять жителей в цивилизацию. И он себя делает, и он второй я, да и разум тут же. И всё легко и свойственно нейронке. И добавлять легко и он очень похож на Айробика. Да. С этого и надо начать. Просто начни с простейшего. Пока текст, код, инет и цифры. А вскоре и гены и химию спокойно. И всё остальное вскоре запросто. То что доктор прописал.

Понимает не только тренинги пока текстовые, но и пытается что то сделать.
Читает One Note и делает что получится!
Создаёт юнитов, делающих то, что и написано в One Note. Пока только какие то обобщения, через автоэнкодер и и конволюшн, а не текст допустим, без выхлопа, а внутрь, это мне и надо.
Это как бы тысячи тебя, но только делающие очень простые действия или как бы намеки на действия, поддействия, недодействия. Шлёпает таких юнитов автоматически, а потом естественный отбор сам выявит, какие делают что то толковое.
'''

In [ ]:
# @title
text = '''
он проводит цепочку невидимую человеком.
1. реализовать 06.02 Модель 18 ОТКРЫТИЕ. НЕЙРОНКА ДУМАЕТ!!!! ВОТ КАК ПОЛУЧИТЬ ДОГАДКИ И ВЫЧИСЛЕНИЯ СВЫШЕ ДАТАСЕТА..mp3
2. !! Обучить нейронку, чтобы Вырезала куски по какому то алгоритму.

Ну не понятно нам, что еда убивает через 5 дней и холестерин во время сна. Ну запретное это. Ну кощунство. Но странно. Или не хочется. или пересекается с главным. Ну еда же. А мы не видим. И еще. Это мы закрываем. А есть вобще не увиденное. Объекты. дороги. качества. А она видит! Это простое 3D! Программируй! Как раз текучка!

а еще есть 1000 людей. А еще есть часто часто. А еще есть рутина, длинные цепочки и феномены не нужные человеку в его выживании и не видимое. есть потоки. есть органы чувств! и много чего!

Сразу. Сразу ТОЛЬКО НЕВЫПОЛНИМОЕ обычным датасетом ЗАДАНИЕ.mp3
Многократные соединения выходов и входов и циклы и снова.mp3
'''

In [ ]:
# @title
text = '''
Поскольку аустаффинг приводит к экономии и повышению производительности предприятия, мы уверены, что за ней будущее. Пока что в России такая услуга кажется чем-то нетрадиционным, но это лишь вопрос времени. Всё-таки, российская экономика и законодательство только недавно сделали возможным развитие этой новой системы найма сотрудников.

Скоро к аутстаффингу будут прибегать и средние, и небольшие компании. На сегодняшний день в штате Rubrain.com числится уже более 6000 аутстафферов, то есть людей, которые находятся «на наших книгах», но фактически работают на различных проектах в разных организациях. С нами сотрудничают такие компании как Mail.ru и «Яндекс», Yota, Mastercard, «МТС», а ещё — десятки зарубежных стартапов. И это только начало.

Объем российского рынка аутстаффинга с 2016-го растет в среднем на 40-45% в год, и сейчас по нашим оценкам составляет около 14 млрд рублей.

В то же время потенциал для развития ещё очень большой — за рубежом услуга куда известнее и популярнее. Так, Citigroup вывела за штат 11 тысяч специалистов, и теперь работает с ними только через провайдера. В Qualcomm через аутстаффинг трудятся отделы в четырех странах. Стартап Worldwide полностью отдал аутстафферам весь комплекс работы над продуктом: программирование, дизайн, маркетинг (SEO/SMM). Для них это был способ быстро получить слаженную и мощную команду, вместо того, чтобы собирать её по крупицам, рискуя упустить рынок. А успешный аутстаффинг из Эстонии в своё время позволил Skype за 2 года перейти от оформления идеи к продаже компании за $2,5 млрд.
'''

In [ ]:
# @title
text = '''
тысяча Я в мою команду! Готовые!!!
Есть готовый код Системы 1 (ohtehtomah) Как теперь этим воспользоваться

• почти готовый видео-голосовой MyChatGPT (говорит, слушает, болтает, смотрит, видит, смотрит камеру!)
• по существу - готовый Я! Говорящий, мудрый, советующий, второй Я для меня же, точнее
	тысяча Я в мою команду!
	○ один - ищет путь.
	○ второй - делает попытки
	○ третий - рассчитывает попытки (как отскочит мяч от стены)
	○ четвертый - реализатор
	○ пятый - мечтатель или скептик
• или считыватель (коллективный интеллект)
• или создатель (например рисует что-то непрерывно, а это же видео, то есть он показывает, строит, созидает!!)
	○ по Prompts можно генерировать не только картинки! Но и видео, и 3D миры и реальное физическое!

У меня почти готов мой высший ИИ!
• ДОучи ChatGPT! Это уже возможно! Он умный, но все рассказывает твоими тренями.
	○ Вместе с ChatGPT уже пора собирать мои нейронки!!!
	○ Вместе с ChatGPT можно превратить тренинги в физическую реализацию, в Цивилизацию!!!

продать.
тебе нужна генерация. Он что-то выдает.
Простейшая, или нет через huggingface
'''

### Полный Огромный (сейчас это 13247 токенов)

In [ ]:
# @title
text = '''
.5......5...................5.........5...55......
вот следующий шаг одной фразой: "Придумай, как ты можешь использовать эту технологию для создания уникального и привлекательного продукта, который будет интересен для пользова

#.     .     .     .     .     .     max_tokens147,temp 0.49, top_p0.30, top_k15, repeat_pen1.15
7......77..6.........6...6..6.6.6....6.......6...6
заголовок, для чего это в проекте: "Новые возможности для людей, которые хотят создавать что-то grandiose и нестандартное!"

#.     .     .     .     .     .     max_tokens119,temp 0.28, top_p0.72, top_k10, repeat_pen1.14
.7...7....6.6..6..6...........66.6..6....6.....6..
укажем одно, для чего это в проекте:
1. Для решения задачи "Стресс" - наша программа поможет справиться с стрессами и тревогой,

#.     .     .     .     .     .     max_tokens113,temp 0.47, top_p0.65, top_k15, repeat_pen1.18
.7.............6.66....6....666...66.6.......66...
Первый шаг к реализации: 1. Начните с того, что вы уже делаете.
2. Определите цель и начать с небольших шагов

#.     .     .     .     .     .     max_tokens122,temp 0.38, top_p0.63, top_k4, repeat_pen1.12
.7..7.........6.66...66........66..........6.6....
вот содержимое улучшенного одной фразой: "Организуй свою жизнь таким образом, чтобы ты мог быстро и эффективно начать работу над тем, что тебе н

#.     .     .     .     .     .     max_tokens110,temp 0.19, top_p0.58, top_k12, repeat_pen1.17
.......................6....6.6..........6..6.....
к следующему шагу, ближе к финалу одна фраза:
"Вот и сделай это, а то ты будешь продолжать сидеть в одной и той же комнате и мечтать

#.     .     .     .     .     .     max_tokens102,temp 0.12, top_p0.35, top_k13, repeat_pen1.16
..6..66.....6........666.....6.6..6.......5.......
структура связей зависимостей этого: 1. Книга - 10 главных героев, каждая из которых имеет свою историю и судьбу.

#.     .     .     .     .     .     max_tokens139,temp 0.49, top_p0.37, top_k18, repeat_pen1.12
77.7.7...66.....6.....6.....6..................6..
Заголовок исходя из того, на каком мы этапе:
"Проект "Сверхскоростная цивилизация": ковер-кросс между искусственным интеллек

#.     .     .     .     .     .     max_tokens106,temp 0.44, top_p0.50, top_k14, repeat_pen1.12
8....8.8..88......7.......7...............7.......
одной фразой, на что это похоже: "Я хочу, чтобы ты был богаче и счастливее!"

Это уже не просто "разработка",

#.     .     .     .     .     .     max_tokens106,temp 0.27, top_p0.27, top_k12, repeat_pen1.14
......6...6..6..6......6......6..6..6.............
Заканчиваем. Финальный заголовок: "Кто хочет стать мастером? Начните с нашего курса!"

#.     .     .     .     .     .     max_tokens150,temp 0.13, top_p0.56, top_k14, repeat_pen1.16
7....7........66....6.6.........6....6.......6....
закреплённый заголовок этой ветки: "Суперники не знают, что я уже сделал 10 000 проектов и 10 миллионов долларов на их

#.     .     .     .     .     .     max_tokens139,temp 0.46, top_p0.48, top_k13, repeat_pen1.16
.......6.66.......6....6....6..........6....66....
Заголовок: СОЗДАНИЕ ИЛИ САМООБУЧЕНИЕ НЯШЕК

#.     .     .     .     .     .     max_tokens136,temp 0.47, top_p0.73, top_k16, repeat_pen1.17
.........6.....6............................6..666
одно, для чего это в проекте, одной фразой: "Этот проект поможет нам понять, как мы можем использовать искусственный интеллект для создания более эффективных и инновацион

#.     .     .     .     .     .     max_tokens111,temp 0.37, top_p0.77, top_k7, repeat_pen1.15
.......7...........7.7..........7...77.......7..7.
в этом тексте главные элементы следующие: - организация жизни для быстрого и эффективного начала работы;
- использование мозга для генерации идей и реше

#.     .     .     .     .     .     max_tokens141,temp 0.12, top_p0.63, top_k16, repeat_pen1.17
....7....7.7.77.7..7.7.......7..6.............6...
на это я скажу только одной фразой: "Я могу создать любой продукт, который вы хотите, и он будет работаться на 10 D".

#.     .     .     .     .     .     max_tokens149,temp 0.11, top_p0.26, top_k6, repeat_pen1.15
..........5....555.........5.5.55.5....55.....555.
укажем одно, для чего это в проекте:
1. реклама на сайте, реклама в социальных сетях, реклаma в видео, рекламные блоки,

#.     .     .     .     .     .     max_tokens115,temp 0.29, top_p0.75, top_k10, repeat_pen1.13
.....6....555....55...5...........55.........5.5..
Главные детали тут:
* Написать программу - это не сложно, но нужно знать, как это сделать.
* Записать звук - это также не сложно

#.     .     .     .     .     .     max_tokens131,temp 0.18, top_p0.46, top_k10, repeat_pen1.14
.77....7......7.7.....77......7.7........7.....7..
уникальное из этого одной фразой: "Вот и я сделал!"




#.     .     .     .     .     .     max_tokens116,temp 0.37, top_p0.79, top_k4, repeat_pen1.13
6....5.55.........5.....5..55.5...5........5......
Конечная Цель:
1. Понимание основных принципов работы с OneNote и его возможности для создания интерактивных учебников.

#.     .     .     .     .     .     max_tokens127,temp 0.49, top_p0.46, top_k16, repeat_pen1.17
.......6..5..55.........55..55..5....5......5.5...
одно, для чего это в проекте, одной фразой: "ООП - это ключевое слово, которое поможет нам создавать более понятный код и уменьшить количество повторений

#.     .     .     .     .     .     max_tokens107,temp 0.16, top_p0.76, top_k5, repeat_pen1.13
...7................6.6.....6....6.6.6...6...6....
Всё это одним словом:
1. Пробить + понарошku + цивилизацию + продажа+ обертка+ сервер.

#.     .     .     .     .     .     max_tokens148,temp 0.19, top_p0.22, top_k14, repeat_pen1.16
..7...7............6...6...6.6..6.6...............
Конечная Цель:
1. Создать инструменты для моделирования поведения людей.
2. Обучать людям использованию инструментов

#.     .     .     .     .     .     max_tokens114,temp 0.32, top_p0.35, top_k11, repeat_pen1.13
................6..6.6...6...6........6...65......
структура связей зависимостей этого:
1. Я - это ваш мозг.
2. Я могу создавать произведения, которые вызывают воображение.

#.     .     .     .     .     .     max_tokens124,temp 0.48, top_p0.52, top_k11, repeat_pen1.17
........6.......66..66..............6..6.....6....
одной фразой, на что это похоже: "Я знаю, что ты можешь делать это лучше, но я хочу, чтобы ты делал это таким образом".

#.     .     .     .     .     .     max_tokens126,temp 0.47, top_p0.21, top_k9, repeat_pen1.17
......6..6...5.5............5....55...5...555..5..
дерево структуры всего проекта: 1. ООП (Объемная Объектно-Ориентированная Programma)
2. Реакция

#.     .     .     .     .     .     max_tokens104,temp 0.37, top_p0.30, top_k9, repeat_pen1.14
8.7777........6.6......6......6.....66.66..6.6....
на основе этих записей и набросков главная фраза: "Сверхскоростная цивилизация - это моя цель!"

#.     .     .     .     .     .     max_tokens105,temp 0.13, top_p0.62, top_k9, repeat_pen1.16
...6.........6......6.6....6..666...6....6..666..6
дерево структуры всего этого: 1. Языки программирования
2. Описание задачи
3. Решение
4. Код
5. Ответы

#.     .     .     .     .     .     max_tokens133,temp 0.22, top_p0.74, top_k7, repeat_pen1.17
.6.............6...6...6.5.......5...5..........5.
всё очень коротко, одной фразой из двух слов: "Я создаю кластеры"




#.     .     .     .     .     .     max_tokens134,temp 0.38, top_p0.23, top_k19, repeat_pen1.17
...............6.....66....6..6..............6..66
уникальное из этого одной фразой: "Я знаю, что ты можешь создать на основе того, что ты уже сделал".

#.     .     .     .     .     .     max_tokens150,temp 0.44, top_p0.48, top_k17, repeat_pen1.17
.....66...66666......6......6...6.66......666.....
дерево этого: 1) Списки
	* 1) список для тех, кто хочет научиться программированию
	*  2) список тех,

#.     .     .     .     .     .     max_tokens128,temp 0.47, top_p0.23, top_k13, repeat_pen1.18
......6.6....6.6.........66......5.......5........
Один следующий пункт:
1. Определите цель для объекта (например, "сверхскоросстная циviлизация"),
2. Создайте игровую ситуа

#.     .     .     .     .     .     max_tokens145,temp 0.11, top_p0.22, top_k10, repeat_pen1.14
.....7........6....6.......6........6.......66....
переходим к самому главному:
1. Кто ты?
2. Что ты делаешь?
3. Зачем ты это делаешь

#.     .     .     .     .     .     max_tokens102,temp 0.44, top_p0.60, top_k6, repeat_pen1.17
.....6.6...........55..........5.....55..5.....5..
Один ответ на этот вопрос:
"Я могу создавать кластеры, но я не могу определиться с тем, какой идеей я должен развивать."

#.     .     .     .     .     .     max_tokens128,temp 0.36, top_p0.27, top_k19, repeat_pen1.17
...7.....77...........6.....66..6....6.6..........
Главные детали, по одному с новой строки: - Я создаю кластры
- Я пишу код
- Я решаю задачи
- Я учитываю ошибки

#.     .     .     .     .     .     max_tokens107,temp 0.13, top_p0.21, top_k6, repeat_pen1.17
.....7...6..............6..6.6......6.6.6.6...6...
Основные сущности из всего этого:
1. Начните с простого, приятного, надежды для богатого, элиты.
2. Затем, постепенно, по мере развития идеи

#.     .     .     .     .     .     max_tokens108,temp 0.35, top_p0.65, top_k7, repeat_pen1.13
..6..6.........6......6.6.......666.......555.....
к следующему этапу ближе к финалу выполнения нашей задачи одна фраза: "Вот и все, теперь нам нужно делать".




#.     .     .     .     .     .     max_tokens134,temp 0.36, top_p0.55, top_k6, repeat_pen1.12
........6..............6.....66.......6.....6...6.
вот содержимое улучшенного одной фразой: "Сделай мои блоков-чудо, я создаю кластры"



#.     .     .     .     .     .     max_tokens109,temp 0.46, top_p0.28, top_k6, repeat_pen1.16
...6.6......5.55...55.5......5.5...............55.
Одна Конечная Цель:
"Создать идеальный мир для всех живущих существ на Земле!"

#.     .     .     .     .     .     max_tokens118,temp 0.48, top_p0.40, top_k7, repeat_pen1.17
...6.........66......6.....6....66.6.6.666....6..6
единственный ответ на этот вопрос: "Куда ты будешь бежать?"

Этот ответ даст тебе возможность начать создавать свою соб

#.     .     .     .     .     .     max_tokens142,temp 0.37, top_p0.52, top_k20, repeat_pen1.15
....6..6.6..................6........6.....6..6..6
заголовок на эту тему: "Как я наутил себя забывать о том, что я не могу делать"

#.     .     .     .     .     .     max_tokens121,temp 0.18, top_p0.39, top_k14, repeat_pen1.16
.7....7.7..77.................7.77..........66..6.
Заголовок где мы сейчас:
"Кластеры"

"Я создаю кластеры"

Примеры:

1. Кластер "Современные технологи

#.     .     .     .     .     .     max_tokens116,temp 0.42, top_p0.57, top_k11, repeat_pen1.16
7.........................666......6....6.....6..6
заголовок на эту тему: "Как научиться программированию, не разбиваясь на нервы?"

Описание:
В этом блоге мы расскажем вам, как научи

#.     .     .     .     .     .     max_tokens145,temp 0.35, top_p0.64, top_k18, repeat_pen1.14
.7.7....77.....7.............7..7...7.77....7.....
чего автор добивается:
1. Создать систему, которая может обучаться и развиваться самостоятельно, используя различные источники информации и технологии.

#.     .     .     .     .     .     max_tokens143,temp 0.31, top_p0.64, top_k11, repeat_pen1.13
....6.6......6.6.6..6...6..6..6.66...6............
Сократим все в 2 коротких заголовка: 1. Не бойтесь ошибаться, а развивайте свои навыки.
2. Начните с простых заданий

#.     .     .     .     .     .     max_tokens116,temp 0.31, top_p0.23, top_k7, repeat_pen1.17
......7.7.................7777.7.7.7..6..........6
Основные сущности из всего этого: 1. Простота.
2 .Легкость.
3. Ясность.
4. Понятность

#.     .     .     .     .     .     max_tokens101,temp 0.28, top_p0.68, top_k13, repeat_pen1.16
...55...........55...5..5........5......4..44..4..
Первый шаг к реализации: - Понимание того, что мы делаем и зачем.
- Определение целей и задач.

#.     .     .     .     .     .     max_tokens100,temp 0.14, top_p0.49, top_k13, repeat_pen1.12
........6.....6...................6..666.......6..
заголовок места, которое между текущим положением и следующим:
ПРОЕКТ "СОЗДАТЬ IDEAL WORLD"



#.     .     .     .     .     .     max_tokens143,temp 0.43, top_p0.30, top_k12, repeat_pen1.13
.....6...6..6..6.......6..6...6......6.6...6......
на основе этих записей и набросков главная фраза: "Приключение с высоким паттерном!"

#.     .     .     .     .     .     max_tokens106,temp 0.45, top_p0.70, top_k19, repeat_pen1.17
.......77......776.......6...6.6........6.6..6....
дерево проекта:
1. Поисковая система для решения задачи.
2. Решение задачи.

#.     .     .     .     .     .     max_tokens148,temp 0.25, top_p0.59, top_k8, repeat_pen1.13
..777.7.77......6.....6.....6.6...6..6.....66.6...
структура связей зависимостей этого: 1. Сверхциviлизация
2. Свершко скоростная цивилизация
3. Если вы хотите, чтобы ваши дети

#.     .     .     .     .     .     max_tokens139,temp 0.28, top_p0.78, top_k16, repeat_pen1.17
..7............6..........6...6.6..66........6....
вот содержимое улучшенного одной фразой:
"Вот и я сделаю!" ( file:///D:/One/mp3/1%20сек/тема%

#.     .     .     .     .     .     max_tokens107,temp 0.15, top_p0.50, top_k9, repeat_pen1.13
6.......5....5..5..5.....55.................555..5
всё очень коротко, одной фразой из двух слов:
* создать систему самоучивания;
* автоматической генерации текста;
* использовать различные источник информации

#.     .     .     .     .     .     max_tokens102,temp 0.26, top_p0.36, top_k15, repeat_pen1.17
7.7.7.7...7..7.............6....6......6....6..6..
одно следующее действие:
1. Считывает текст из файла или из базы данными.
2.. Если есть цели, то они должны быть считан

#.     .     .     .     .     .     max_tokens143,temp 0.37, top_p0.69, top_k7, repeat_pen1.15
....6.6....6.....66........6...6..55.....5..5.....
в этом тексте главные элементы следующие:
1. Нейронные сетики - основание для ускорению цивилизаций.
2. Реализация - фабрика, завод, научно

#.     .     .     .     .     .     max_tokens126,temp 0.31, top_p0.37, top_k6, repeat_pen1.18
......7..........6...6.....66......6....6.....6...
дерево структуры всего проекта: 1. Описание проекта
2. Цели и задачи
3. Тематика
4. Сценарии
5. Визуаль

#.     .     .     .     .     .     max_tokens132,temp 0.22, top_p0.25, top_k11, repeat_pen1.13
.777..6..6.666...............6.....6666..6........
укажем одно, для чего это в проекте: - реализация чата с искусственным интеллектом на основе технологии deep learning;
- использование модели для генерации

#.     .     .     .     .     .     max_tokens137,temp 0.36, top_p0.71, top_k19, repeat_pen1.17
.....6..6........6....66.6.............6......6...
дальше я запишу одну фразу: "Я хочу создать идеальный мир"
и он будет отвечаться на эту фразу

#.     .     .     .     .     .     max_tokens119,temp 0.49, top_p0.62, top_k14, repeat_pen1.15
.7...77...7........7.7................7.7..7766...
чего автор добивается:  1. Увеличение количества заказов на разработку ИИ-продуктов.
2 . Увеличение качества разработки ИИ-

#.     .     .     .     .     .     max_tokens123,temp 0.30, top_p0.20, top_k4, repeat_pen1.14
..7..........7......66.6........6..........6.....6
Вот что позволит это улучшать одной фразой: "Сделайте свою жизнь лучше, используя AI".
Это просто и логично

#.     .     .     .     .     .     max_tokens146,temp 0.20, top_p0.25, top_k8, repeat_pen1.16
.88......7...........77.......77.............7....
дерево проекта:
- реализация нейронных сеток
- создание ускоренной цивилизации
- интересная жизнь

#.     .     .     .     .     .     max_tokens143,temp 0.31, top_p0.60, top_k18, repeat_pen1.15
7.7..7.7..7...7.7..7.........7.........66....6.6..
заголовок этапа, который между текущим положением и готовностью: "Преодоление препятствий на пути к созданию Института Искусственного Инtellecta"

#.     .     .     .     .     .     max_tokens130,temp 0.13, top_p0.32, top_k11, repeat_pen1.15
....7..7.....66........6.........6..6...6.6..66.6.
Из этого следует одно: - Я хочу создавать идеальный Мир
- Я могу создавать идеалы, но я не могу их реализовать

#.     .     .     .     .     .     max_tokens141,temp 0.44, top_p0.54, top_k5, repeat_pen1.16
.........7...7....7.7.......7........7.7..6......6
вот следующий шаг одной фразой:
"Я создам идеальный Мир"

и он ответит на эту Фразу

#.     .     .     .     .     .     max_tokens150,temp 0.30, top_p0.46, top_k16, repeat_pen1.15
........6...................5........5...5..5.....
вот основная часть:
1. Сделайте жизнь себя лучше, использованием AI.
2. Придумай задачи-продюкт с азартом в творчестве

#.     .     .     .     .     .     max_tokens103,temp 0.22, top_p0.62, top_k8, repeat_pen1.13
.7..............77........7......7..7.........6.6.
Главные детали, коротко, по два слова на каждое:
1. Проект (what)
2. Цель (goal)
3. Темatica (theme)

#.     .     .     .     .     .     max_tokens127,temp 0.15, top_p0.22, top_k18, repeat_pen1.18
....6.........66....6.6......6.....5........55....
Главное в этом короткими фразами из двух слов, каждое с новой строки:
1. Создай свой продукт.
2. Найдите продажников (рекламируйте).

#.     .     .     .     .     .     max_tokens113,temp 0.27, top_p0.64, top_k13, repeat_pen1.14
........66..6....6...........6.........5..........
Основные сущности из всего этого: - Создавать идеального мира
- Реализовывать идеалов
- Продавать продукты
- Не откладывать

#.     .     .     .     .     .     max_tokens110,temp 0.20, top_p0.79, top_k7, repeat_pen1.12
.............66........66......6....6.....6...66..
Всё это одной фразой: "Я хочу, чтобы все были живы и здоровы, и я буду делать все возможное, чтобы это произошло".

#.     .     .     .     .     .     max_tokens101,temp 0.28, top_p0.50, top_k5, repeat_pen1.13
.....6.........6....5..........5...5.5........555.
дерево структуры всего этого: 1. картина мира
2. текст
3. связки
4. объекты
5. раскаленные облака

#.     .     .     .     .     .     max_tokens148,temp 0.34, top_p0.60, top_k15, repeat_pen1.18
....7.....777.77..7....77...7.7............77.7...
Один следующий пункт:
Это уже не простое копирование, а полноценный механизм, который может работать в любой области.

#.     .     .     .     .     .     max_tokens125,temp 0.39, top_p0.35, top_k4, repeat_pen1.15
6................................5..5.5........5..
Главное в этом короткими фразами из двух слов, каждое с новой строки:
- Сверхскоростная цивилизация
- Светлое будущее
- Невероятные возможности

#.     .     .     .     .     .     max_tokens105,temp 0.11, top_p0.71, top_k6, repeat_pen1.14
..6..6.6.6...66..........6......6..........6...6.6
Сократим все в 2 коротких заголовка: 1. СверхСCOOP - Сверхскоростная цивилизация для всех.
2. Сверхигра - Сверхигрографические приключения

#.     .     .     .     .     .     max_tokens117,temp 0.31, top_p0.31, top_k15, repeat_pen1.17
.......7...7....6.........6.6.......6....6...6....
к следующему шагу, ближе к финалу одна фраза: "Вы можете создавать свою собственную цивилизацию!"

#.     .     .     .     .     .     max_tokens117,temp 0.29, top_p0.40, top_k14, repeat_pen1.14
....6.6...6...6..........6..6..........6.66.......
одно следующее действие:
Просто запусти в работу свою армию инженерных-реализовщиков и начните создавать свою страну

#.     .     .     .     .     .     max_tokens127,temp 0.15, top_p0.49, top_k19, repeat_pen1.14
7.6.................6.6.6..6...............6......
заголовок по этой теме: "Проект "Эволюция Creativity": создание инновационных продуктов с использованием AI для развития человеческого творчества"

#.     .     .     .     .     .     max_tokens111,temp 0.32, top_p0.66, top_k11, repeat_pen1.18
......76...6.6....6.6...6.6...66....66..6.........
Главный инструмент здесь:
* **Диктафон** - для того чтобы записать свои идеи и планы на будущее.

#.     .     .     .     .     .     max_tokens132,temp 0.12, top_p0.68, top_k4, repeat_pen1.16
...6.6.6.6.6...5.........55.5...................5.
Из всего этого вывод:
* Я хочу, чтобы все были жилы и здорovy, и я будет делать все, чтобы это случилось.



#.     .     .     .     .     .     max_tokens113,temp 0.26, top_p0.60, top_k20, repeat_pen1.15
.7..........6....6.6..666......66......6..6...6...
одно, для чего это в проекте, одной фразой: "Вы будете создавать свои собственные идеи, а мы поможем вам их реализовать!"

#.     .     .     .     .     .     max_tokens136,temp 0.29, top_p0.37, top_k6, repeat_pen1.13
............6...55..5.....5....55.5..5...55...5..5
Вот что позволит это улучшать одной фразой: "Для того чтобы создать свою собственую цивилизацию, вы должны..."

Это будет более конкретно и интересно

#.     .     .     .     .     .     max_tokens115,temp 0.28, top_p0.70, top_k16, repeat_pen1.15
.........6.6...............6.....6...6.....6..6...
какая цель?
1. Обработка информации
2. Управление временем
3. Рациональное мышление

#.     .     .     .     .     .     max_tokens111,temp 0.31, top_p0.56, top_k11, repeat_pen1.13
7...7.....7...6.......6.....6....66.......6.6....6
единственный ответ на этот вопрос:
"Я могу создавать свою own civilization!"

или "I can create my own civilization!".

#.     .     .     .     .     .     max_tokens111,temp 0.23, top_p0.59, top_k20, repeat_pen1.12
.6..6....6.........5....5...5..55555.5...5.5......
Из всего этого главные действия: 1) Реализация первой части модели (людей)
2) Создание моделей для чтение текста и видео

#.     .     .     .     .     .     max_tokens130,temp 0.33, top_p0.77, top_k18, repeat_pen1.14
7....7...7..6.......66............................
Один следующий пункт:
"Эволюция creativity" - это процесс создания инновационной продукции с использованием искусственного интеллекта для развития human creativity

#.     .     .     .     .     .     max_tokens121,temp 0.14, top_p0.80, top_k7, repeat_pen1.12
.6..6.6..6...6...6....6..5.......5.....5..........
Главные детали тут:
1. Сверхцель - это цель, которая не может быть достигнута обычными методами, требует новых идей, новых подходов, новых

#.     .     .     .     .     .     max_tokens126,temp 0.20, top_p0.72, top_k13, repeat_pen1.14
...7..........6.....66..................6....6....
заголовок по этой теме: "Супер-машинка для создания супер-текстов"




#.     .     .     .     .     .     max_tokens122,temp 0.31, top_p0.52, top_k18, repeat_pen1.14
.7.......6..6......6...6....666...................
заголовок по этой теме: "Супермодель для ИИ: создание и реализация своих идей"



#.     .     .     .     .     .     max_tokens125,temp 0.25, top_p0.58, top_k17, repeat_pen1.12
.77.7.7........7............6..............6......
заголовок места, которое между текущим положением и следующим: "Нейронные сетики, как грыбы, возникли то тут то там, и я уже стал их использовать для обучения"

#.     .     .     .     .     .     max_tokens112,temp 0.49, top_p0.64, top_k13, repeat_pen1.18
.7....777..7.7...........7..7..7.7....7..7........
дерево этого:
1. реализация людей
2. создание моделя для чтения текста и просмотра видео

#.     .     .     .     .     .     max_tokens139,temp 0.13, top_p0.52, top_k18, repeat_pen1.13
.....6....5..5....................5.5..5..........
вот основная часть:
[/code]  Welcome to the most exciting and innovative AI-powered writing tool - PitchChamp! 🚀

#.     .     .     .     .     .     max_tokens145,temp 0.18, top_p0.36, top_k8, repeat_pen1.16
7......66....6...6.....................6...6.6.66.
в этом тексте главные элементы следующие:
- сверхцели (super goal) - достижение цели, которую не может достичь обычным способом, требуется новые иде

#.     .     .     .     .     .     max_tokens130,temp 0.17, top_p0.44, top_k11, repeat_pen1.14
...66......5.5....5.....5....5.......5........5.5.
Один ответ на этот вопрос:
\Q:

#.     .     .     .     .     .     max_tokens102,temp 0.31, top_p0.42, top_k11, repeat_pen1.14
.7.7...7......7..7.7........7.....7....77.7.......
Из этого следует одно:
Конечно, я могу рассказать о том, как я планирую развивать свой проект, но я не могу

#.     .     .     .     .     .     max_tokens138,temp 0.30, top_p0.43, top_k20, repeat_pen1.15
.......66..6......6.................5...5.........
к следующему этапу ближе к финалу выполнения нашей задачи одна фраза:
"Я не могу сказать, что я счастлив, но я чувствую себя более комфортно

#.     .     .     .     .     .     max_tokens102,temp 0.17, top_p0.33, top_k15, repeat_pen1.18
.6........4....4..444.........4...44.4.44...3.....
дерево этого: 1. Я - Император
2. Я - Бессмертний
3. Я - Помню тыша

#.     .     .     .     .     .     max_tokens103,temp 0.18, top_p0.26, top_k6, repeat_pen1.16
77..7.7............6.....6....6..6....6...6..66...
Из этого следует одно:
Вы можете начать с того, что у вас есть идеи, которые вы хотели бы реализовать, но не знаете, как

#.     .     .     .     .     .     max_tokens118,temp 0.35, top_p0.22, top_k11, repeat_pen1.14
...................6.....6..66...5...5......5.5...
Что хочет автор:
1. Построить свою жизнь таким образом, чтобы она была более интересной и продуктивной.

#.     .     .     .     .     .     max_tokens144,temp 0.26, top_p0.73, top_k8, repeat_pen1.16
.......7...................7.7......7.....7.......
Главные детали, по одному с новой строки:
1. Супермодель Для ИИ
2. Костяк, живой, золотой
3. Продать то, что есть

#.     .     .     .     .     .     max_tokens124,temp 0.43, top_p0.68, top_k15, repeat_pen1.16
.............6.6.6.6........6..................666
переходим к самому главному:
- какой у меня продукт?
- какие возможности я предлагаю?
- как я их реализую?

#.     .     .     .     .     .     max_tokens108,temp 0.32, top_p0.63, top_k14, repeat_pen1.17
7.77.......77.....6.6.6.6.66....6........666..6.6.
к следующему шагу, ближе к финалу одна фраза: "Я уже сделал то, что мог, но мне нужно больше!"

#.     .     .     .     .     .     max_tokens132,temp 0.20, top_p0.27, top_k11, repeat_pen1.15
6....6...........5.5....5..55.5....5...........555
на основе этих записей и набросков главная фраза: "Плотно и не расслабливо!"

#.     .     .     .     .     .     max_tokens104,temp 0.28, top_p0.29, top_k7, repeat_pen1.15
6.......66.6...6..................5.5......5..5...
Из всего этого главные действия: 1. Запишите текст, который вы будете реализованным.
2. Определите ключевые слова и фразы, которые

#.     .     .     .     .     .     max_tokens150,temp 0.34, top_p0.63, top_k16, repeat_pen1.13
8...........77......7....7....7..77......77.7..7..
Один следующий пункт:
Этот план может быть использован для любых задач, связанных с моделированием и искусственным интеллектом

#.     .     .     .     .     .     max_tokens142,temp 0.39, top_p0.73, top_k19, repeat_pen1.14
7.7.7777........77..7..6......6.....6....66....6..
заголовок на эту тему: "Собрать супермодель для ИИ: от начала до конца"

описание:
"В этом проекте мы будем собирать супер

#.     .     .     .     .     .     max_tokens113,temp 0.37, top_p0.67, top_k20, repeat_pen1.16
.77.7...77.....7..7...7.7..7....7......7.....7.7..
Заканчиваем. Финальный заголовок:
"Я всегда был и остаюсь в процессе развития, ищу новые способы достижения целей, ищу свои возможности

#.     .     .     .     .     .     max_tokens112,temp 0.33, top_p0.30, top_k16, repeat_pen1.17
..7..7...7.7.......7......7....7...7.7....7.7.....
уникальное из этого одной фразой: "Я могу научить любой нейронный сетей добиться высоких результатов, но для этого нужно поставить задачи, которые он сможет

#.     .     .     .     .     .     max_tokens122,temp 0.49, top_p0.66, top_k12, repeat_pen1.18
.7............7...6......6..66...6..6.......66....
дерево структуры всего этого: 1. Поставь цель.
2. Поставль картинку (или видео).
3. Поставй условие.

#.     .     .     .     .     .     max_tokens112,temp 0.13, top_p0.30, top_k5, repeat_pen1.14
76....6.6....6.6.6.66.........6.6.....5....5.55.5.
одной фразой, на что это похоже: "Вы хотите сделать что-то круто и интересное, но не знаете, как начать?"

#.     .     .     .     .     .     max_tokens148,temp 0.32, top_p0.32, top_k13, repeat_pen1.13
6...6..............55..5...5........5.....55......
одно следующее действие:
1. Загрузите ImageNet и обученные NN (Neural Networks) для классификации изображений.

#.     .     .     .     .     .     max_tokens120,temp 0.19, top_p0.40, top_k13, repeat_pen1.13
...7....6...6...6.........66........6...6.66...6.6
Из этого следует одно:
1. Надо определиться c первым деコдером;
2. Надо создать Юниты, которые будут сами плодовиться

#.     .     .     .     .     .     max_tokens113,temp 0.33, top_p0.30, top_k17, repeat_pen1.18
.77.....6...........6...........6.6...6....6.....6
Что хочет автор: 1. Показать, как создать нейронную сеть, которая может выполнять различные задачи, не требуя большого

#.     .     .     .     .     .     max_tokens113,temp 0.48, top_p0.65, top_k20, repeat_pen1.14
...8.......77....7.....7..7..7.......7....7...77..
Заголовок исходя из того, на каком мы этапе: "Как я научился создавать нейронные сети с нуля"

Вот некоторые моменты, которые могут быть интересны

#.     .     .     .     .     .     max_tokens132,temp 0.29, top_p0.53, top_k13, repeat_pen1.14
......7.................7.7....7.77.......7...7...
дерево проекта: 1. Понимание того, что происходит внутри нейронной сети.
2. Создание визуализированного представления проекта

#.     .     .     .     .     .     max_tokens147,temp 0.21, top_p0.38, top_k19, repeat_pen1.16
7...6..6.......6....6.....6...6..............66...
заголовок места, которое между текущим положением и следующим: "Как научиться делать рекламу на YouTube без знаний программирования и графического дизайна"

#.     .     .     .     .     .     max_tokens126,temp 0.16, top_p0.28, top_k4, repeat_pen1.14
...777.66..............6.......6.66...6....6......
Одна Конечная Цель:
Давайте создадим вместе инновационный проект, который будет приносить богатство и счастья

#.     .     .     .     .     .     max_tokens107,temp 0.17, top_p0.49, top_k12, repeat_pen1.16
.......7...777................6.....66666.6.....6.
дерево структуры всего проекта:
1. Моделирование (модели)
2. Кодирование (код)
3. Тестирование (тесты)

#.     .     .     .     .     .     max_tokens137,temp 0.18, top_p0.71, top_k16, repeat_pen1.17
.....6.........6.6...6.6.........6...6..6.6..6....
Первый шаг к реализации:
Определите, какие функциональные возможности вам необходимы для начала работы с проектом.

#.     .     .     .     .     .     max_tokens119,temp 0.31, top_p0.73, top_k10, repeat_pen1.15
...6.........6...6.66......6.....6.....6..........
дальше я запишу одну фразу: "Сделай так, чтобы мой мозг был настроен на создание!"

#.     .     .     .     .     .     max_tokens115,temp 0.17, top_p0.73, top_k8, repeat_pen1.13
...6..55..................5...5...5.....5.....5...
Вот что позволит это улучшать одной фразой: "Ты можешь делать это, ты можешь делавать это, ты уже делаешь это".

#.     .     .     .     .     .     max_tokens125,temp 0.45, top_p0.58, top_k11, repeat_pen1.16
6......66....5.....5..5.....55.....5.5.5.5........
Вот что позволит это улучшать одной фразой: "Я не могу, я не знаю, я не могу делать".

Этот текст является компиляцией различных советов и рекомендаций

#.     .     .     .     .     .     max_tokens105,temp 0.22, top_p0.26, top_k5, repeat_pen1.16
....5.........5......5..........5.5.55...........4
Вот что позволит это улучшать одной фразой: "Я не могу помнить, что делал вчера, но я всегда могу делать сегодня".

#.     .     .     .     .     .     max_tokens131,temp 0.11, top_p0.49, top_k13, repeat_pen1.12
..........6..6..6........6.....6..6...............
единственный ответ на этот вопрос: "Какой выбор?"

#.     .     .     .     .     .     max_tokens142,temp 0.48, top_p0.61, top_k8, repeat_pen1.13
..7....7..7.7..7.......7.7...7...7...77........77.
дерево структуры всего проекта: 1.  Нейронки
2.  Потоки
3.  Высокие Уровни
4.  Автохвосты

#.     .     .     .     .     .     max_tokens113,temp 0.24, top_p0.49, top_k13, repeat_pen1.15
7.......6.6.6..6........6...6.........6........6.6
вот основная часть:
1.  Я буду создавать систему, которая поможет людям создавать искусственный интеллек.
2.  Я называю эту систему "Gorodok Kompani"

#.     .     .     .     .     .     max_tokens146,temp 0.38, top_p0.63, top_k18, repeat_pen1.17
7...7......77.7777.........7........6......66.....
всё очень коротко, одной фразой из двух слов: "Виртуальная страна, где алгоритмические модели создают новый мир".

#.     .     .     .     .     .     max_tokens102,temp 0.45, top_p0.26, top_k13, repeat_pen1.13
.77.77...........6....6.............66..666..66..6
одной фразой, на что это похоже:
"Я хочу создать систему, в которой люди будут работать вместе, используя свои знания и умения, чтобы достичь целей, которые не могу

#.     .     .     .     .     .     max_tokens106,temp 0.13, top_p0.53, top_k14, repeat_pen1.18
.7...................6......6................6.66.
дальше я запишу одну фразу:
"Продажа идеи через ChatGTPT"

и будем разбираться в ней

#.     .     .     .     .     .     max_tokens110,temp 0.37, top_p0.59, top_k9, repeat_pen1.15
...............6....66.6.....................6.6..
на основе этих записей и набросков главная фраза: "Переход к коллективному разуму"

17:47

Этот текст был создан с помощью

#.     .     .     .     .     .     max_tokens123,temp 0.12, top_p0.23, top_k14, repeat_pen1.16
..............6.6...66..6...666........66.6...6..6
дерево этого: 1.Сценарий
2.Выбор варианта
3.Планирование
4.Изготовление

#.     .     .     .     .     .     max_tokens147,temp 0.18, top_p0.28, top_k17, repeat_pen1.14
.....5..5........55..5..55.................5..5...
в этом тексте главные элементы следующие:
1. рерайт;
2. советы для создания контента;
3. технические рекомендования

#.     .     .     .     .     .     max_tokens119,temp 0.44, top_p0.61, top_k9, repeat_pen1.14
........6.....6...........................6.....66
перепишем это короче, одной фразой:
"Я создам Gorodok Kompanii, где люди создают искусственный интелект, а я помогую им сделать это более легким и интерес

#.     .     .     .     .     .     max_tokens118,temp 0.33, top_p0.48, top_k20, repeat_pen1.17
..7.....6...66..6.....6....6..66.6....6....6....6.
Один следующий пункт:
* Обратите внимание, что все эти идеи могут быть использованы для создания искусственного интеллек

#.     .     .     .     .     .     max_tokens106,temp 0.50, top_p0.27, top_k17, repeat_pen1.14
8.8888..........7...................7..77....7..7.
укажем одно, для чего это в проекте:
* создание системы, которая будет создавать искусственные интеллекты;
* разработка технологий, которые позволят создавать такие систем

#.     .     .     .     .     .     max_tokens111,temp 0.37, top_p0.24, top_k12, repeat_pen1.16
.666..........6......65........................55.
дерево структуры всего этого:
1. Нейронные сетики
2. Квантовые компьютеры
3. Компьютерная графика

#.     .     .     .     .     .     max_tokens145,temp 0.13, top_p0.28, top_k10, repeat_pen1.14
6..6..6....................6.6...6.6..............
структура связей зависимостей этого: 1️⃣ президент
2️⃵ премьер-министр
3️⃶ министры

#.     .     .     .     .     .     max_tokens112,temp 0.42, top_p0.49, top_k8, repeat_pen1.18
..77..7..6.......6.......6...66...............6...
заголовок на эту тему: "Нейронные сетики: от фабрики до научного города"

Описание:
Это статья о том, как нейронные

#.     .     .     .     .     .     max_tokens105,temp 0.32, top_p0.67, top_k10, repeat_pen1.16
.77.7................77...7.......7...7...........
дерево структуры всего проекта:
1. Нейронки
2. Кодим
3. Проекты
4. Скрипты

#.     .     .     .     .     .     max_tokens127,temp 0.43, top_p0.68, top_k7, repeat_pen1.13
66.6..6.6.........6...6..6..66.......6...666..6..5
Главные детали тут:
1. Слушает - в Памяти, Конволюционная система.
2. Блоки с several LR.

#.     .     .     .     .     .     max_tokens129,temp 0.17, top_p0.67, top_k20, repeat_pen1.18
.6...66.66...6..6......6................5....5..5.
Заголовок где мы сейчас: "Создание модели для анализа текстовых данных с использованием нейронных сетей"

#.     .     .     .     .     .     max_tokens141,temp 0.38, top_p0.61, top_k19, repeat_pen1.13
..77............7..........6....666...66......66.6
дерево этого: 1. Понятие о том, что происходит в мозге при воспринимании текста
2. Обучение модели на основе дан

#.     .     .     .     .     .     max_tokens138,temp 0.45, top_p0.79, top_k19, repeat_pen1.17
..6.6..6.6...6....6..6.6....6.......6..6..........
Сократим все в 2 коротких заголовка: 1. СверхСコростной Цивиализации (SSC)
2. Наши знания и навыки

#.     .     .     .     .     .     max_tokens113,temp 0.23, top_p0.70, top_k4, repeat_pen1.17
...6..6.........6..66......6.........66...66......
общая структура этого: 1. введение
2. основной текст
3. эпизодические память
4. долговременная

#.     .     .     .     .     .     max_tokens118,temp 0.37, top_p0.43, top_k17, repeat_pen1.14
77........66.6...6..6.....6........666....6..5....
Вот что позволит это улучшать одной фразой:
"Они могут использовать свои знания и навыки для создания более эффективных и продуктивных моделей, а также для улучшения

#.     .     .     .     .     .     max_tokens139,temp 0.36, top_p0.60, top_k18, repeat_pen1.13
..7..........7.6....6..6..66.........6666.6.......
заголовок этапа, который между текущим положением и готовностью: "Я могу создать хороший скрип на 10D, но я не могу создать все, что нужно для реального проекта".

#.     .     .     .     .     .     max_tokens125,temp 0.34, top_p0.76, top_k12, repeat_pen1.13
........777......7.77..77..........7..............
Главные детали тут: 1. Прикрепить контроллеры
2. Обучать няшкам параметров
3. Не считать с

#.     .     .     .     .     .     max_tokens135,temp 0.19, top_p0.45, top_k6, repeat_pen1.16
.7.7.......6.............6.6..666.....655.....5...
дерево проекта: 1) Создание блочной системы для обучения модели на текст.
2) Обучение без учителей.

#.     .     .     .     .     .     max_tokens121,temp 0.41, top_p0.72, top_k15, repeat_pen1.17
...7..77.7.......6.......666........66...6..6.....
Заголовок где мы сейчас: "Обучение без Учителя на основе High-Level IMDB и Similar"




#.     .     .     .     .     .     max_tokens115,temp 0.35, top_p0.33, top_k5, repeat_pen1.14
..................7...7.....7.........7...........
дерево проекта: 1. Настройка системы
2. Обучение системы
3. Оценка качества
4. Оптимизация

#.     .     .     .     .     .     max_tokens124,temp 0.24, top_p0.44, top_k11, repeat_pen1.12
...............55....5..........55.....5.....5.5.5
на это я скажу только одной фразой: "Выберите ваш путь!"

#.     .     .     .     .     .     max_tokens121,temp 0.17, top_p0.60, top_k9, repeat_pen1.16
...6.....6...........6............6..6........5..5
переходим к самому главному:
1. Установите цель -
---------------------------------------------------------------------------
KeyboardInterrupt                         Traceback (most recent call last)
<ipython-input-69-824b1ee6c52f> in <cell line: 1>()
    103     # print('\n```python\n')
    104     with torch.autocast('cuda', dtype=torch.float32):
--> 105         result = bot_answers(prompt, max_tokens=max_tokens, temperature=temperature, top_p=top_p, top_k=top_k, repeat_penalty=repeat_penalty)
    106     # print('\n```')
    107     print(f"\n#.     .     .     .     .     .     max_tokens{max_tokens},temp {temperature:.2f}, top_p{top_p:.2f}, top_k{top_k}, repeat_pen{repeat_penalty:.2f}")

17 frames
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py in special_tokens_map_extended(self)
   1278         set_attr = {}
   1279         for attr in self.SPECIAL_TOKENS_ATTRIBUTES:
-> 1280             attr_value = getattr(self, "_" + attr)
   1281             if attr_value:
   1282                 set_attr[attr] = attr_value

KeyboardInterrupt:
'''